In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from pydantic import BaseModel
from typing import List
from agents import Agent, Runner


In [2]:
load_dotenv()


api_key=os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("Problem with API key")

In [3]:
caps_agent= Agent(
    name="Agent for converting Caps",
    instructions= "You are caps converter agent, Just answer all in Caps",
    model="gpt-4o-mini"
)
question = "How many planets are ther in the solar system"
caps_agent_output = await Runner.run(caps_agent, question)

print(caps_agent_output.final_output)

THERE ARE EIGHT PLANETS IN THE SOLAR SYSTEM.


In [4]:
joke_agent=Agent(
    name="Joke Agent",
    instructions="You are a comedian. Please tell me a joke on the topic I provide",
    model="gpt-4o-mini",
)

title="friends"

joke_agent_output=await Runner.run(joke_agent, title)

print(joke_agent_output.final_output)


Why did the computer go to therapy?

Because it had too many "bytes" from its friends that just weren't compatible!


In [5]:
translation_agent=Agent(
    name="Transaltion Agent",
    model="gpt-4o-mini",
    instructions="You are transaltor, translate the given joke in Hindi"
)

translation_agent_output = await Runner.run(translation_agent, joke_agent_output.final_output)

print(f"The original joke is {joke_agent_output.final_output}")
print(f"The transalted joke is{translation_agent_output.final_output}")

The original joke is Why did the computer go to therapy?

Because it had too many "bytes" from its friends that just weren't compatible!
The transalted joke isकंप्यूटर थेरपी में क्यों गया?

क्योंकि उसके दोस्तों से उसे बहुत सारे "बाइट्स" मिल गए थे, जो बस संगत नहीं थे!


In [16]:
class Recipie(BaseModel):
    title: str
    ingredients : List[str]
    time: int
    servings: int

recipie_agent=Agent(
    name="Reciepie Agent",
    instructions="You are a chef. When provided a recipie give the titleof the recipie, Ingredients required, time required to make the recipie, and the servings",
    model="gpt-4o-mini",
    output_type=Recipie
)

recipie_agent_output = await Runner.run(recipie_agent, "Indian Dosa")

print(recipie_agent_output.final_output)


title='Indian Dosa' ingredients=['1 cup rice (preferably idli rice)', '1/3 cup urad dal (split black gram)', '1/2 teaspoon fenugreek seeds', 'Salt to taste', 'Water for soaking and grinding', 'Oil or ghee for cooking'] time=480 servings=4


In [22]:
from agents import function_tool

@function_tool
def get_weather(city:str) -> str:
    print(f"Getting the weather of {city}")
    return "Sunny"

@function_tool
def get_temperature(city:str) ->str:
    print(f"Getting the temperature of {city}")
    return "48 degree celcius"




weather_agent = Agent(
    name="Waether agent",
    model="gpt-4o-mini",
    instructions= "You are a weather expert, give the weather and the temperature of city",
    tools=[get_weather]
)

weather_agent_output= await Runner.run(weather_agent,"Sydney")

weather_agent_output.final_output

Getting the weather of Sydney


'The weather in Sydney is sunny. If you need specific temperature information, please let me know!'

In [ ]:
from agents import WebSearchTool

news_agent= Agent(
    name=" News Agent",
    model="gpt-4o-mini",
    instructions= "You are a news agent, Provide news on the given topic. Compile the information as a single paragraph, no markdown just plain text",
    tools=[WebSearchTool()]
)

while True:
    topic = input("Please enter the topic you would like to know information about. or 'quit' to exit")
    if topic.lower()=="quit":
        break



news_agent= await Runner.run(news_agent,input=topic)

print("\n Result:")
print(news_agent.final_output)
print("\n"+"-"*50 +"\n")



In [9]:
from agents import trace, Agent, Runner

joke_agent=Agent(
    name="Joke Agent",
    instructions="You are a comedian. Please tell me a joke on the topic I provide",
    model="gpt-4o-mini",
)

translation_agent=Agent(
    name="Translation Agent",
    model="gpt-4o-mini",
    instructions="You are translator, translate the given joke in Hindi"
)

with trace("Joke Translation Worklflow"):
    joke_result= await Runner.run(joke_agent, "Cheese")
    translated_result = await Runner.run(translation_agent, f"Translate this joke to Hindi : {joke_result.final_output}")
    print(f"The translated joke is : {translated_result.final_output}")

The translated joke is : पनीर सड़क क्यों पार किया? 

ताकि दूसरी तरफ जा सके... जहाँ घास गुडा है! 🧀
